In [0]:
%run "/Workspace/databricks_webscraping_football/databricks_football_scraping/databricks/setup/access_azure_datalake_using_service_principal"

In [0]:
from delta.tables import DeltaTable

# ========================================
# CARREGAR DADOS JSON DO AZURE (COM MULTILINE)
# ========================================

df_calendar = spark.read.format("json") \
    .option("multiLine", "true") \
    .load("abfss://calender@scrapingfootball.dfs.core.windows.net/all_calendar.json")

df_results = spark.read.format("json") \
    .option("multiLine", "true") \
    .load("abfss://results@scrapingfootball.dfs.core.windows.net/all_results_incremental.json")

df_statistics = spark.read.format("json") \
    .option("multiLine", "true") \
    .load("abfss://statistics@scrapingfootball.dfs.core.windows.net/incremental_games_statistics.json")

print(f"✅ Dados JSON carregados:")
print(f"   Calendar: {df_calendar.count()} registros")
print(f"   Results: {df_results.count()} registros")
print(f"   Statistics: {df_statistics.count()} registros")

In [0]:
calendar_path = "/Volumes/databricks_webscraping_football_ws/raw/calendar/all_calendar"

if DeltaTable.isDeltaTable(spark, calendar_path):
    delta_calendar = DeltaTable.forPath(spark, calendar_path)
    delta_calendar.alias("target").merge(
        df_calendar.alias("source"),
        "target.id = source.id"
    ).whenMatchedUpdateAll() \
     .whenNotMatchedInsertAll() \
     .execute()
    print("✅ Calendar atualizado (MERGE)")
else:
    df_calendar.write.format("delta").mode("overwrite").save(calendar_path)
    print("✅ Calendar criado pela primeira vez")

# RESULTS
results_path = "/Volumes/databricks_webscraping_football_ws/raw/results/all_results"

if DeltaTable.isDeltaTable(spark, results_path):
    delta_results = DeltaTable.forPath(spark, results_path)
    delta_results.alias("target").merge(
        df_results.alias("source"),
        "target.id = source.id"
    ).whenMatchedUpdateAll() \
     .whenNotMatchedInsertAll() \
     .execute()
    print("✅ Results atualizado (MERGE)")
else:
    df_results.write.format("delta").mode("overwrite").save(results_path)
    print("✅ Results criado pela primeira vez")

# STATISTICS
statistics_path = "/Volumes/databricks_webscraping_football_ws/raw/statistics/all_statistics"

if DeltaTable.isDeltaTable(spark, statistics_path):
    delta_statistics = DeltaTable.forPath(spark, statistics_path)
    delta_statistics.alias("target").merge(
        df_statistics.alias("source"),
        "target.id = source.id"
    ).whenMatchedUpdateAll() \
     .whenNotMatchedInsertAll() \
     .execute()
    print("✅ Statistics atualizado (MERGE)")
else:
    df_statistics.write.format("delta").mode("overwrite").save(statistics_path)
    print("✅ Statistics criado pela primeira vez")